## Set-Up the Environment

In [ ]:
%load_ext autoreload
import sys
sys.path.append("../src")
import pyabf
import slopeTools
import abfTools
import plotTools
import statsTools
import numpy as np
import matplotlib.pyplot as plt

## Load abf files 

In [ ]:
abfFilePaths = [
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804007.abf",
    #R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804024.abf", 
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804030.abf", 
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804043.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804048.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804060.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20804066.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805008.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805029.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20805035.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20811011.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20811021.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20817012.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20831011.abf",
    R"X:/Data/C57/TGOT on PVT/2020-07-28 10nM TGOT on PVT/20831017.abf"
]
#abfFilePaths = abfFilePaths[4:6]

## ABF Holding Current Slope Analysis
* Get holding current from every sweep
* Apply a moving window filter to smooth the data
* Apply a single linear regression in the baseline period (0-5min) to get the slope of baseline.
* Generate multiple linear regression on the segregated data during drug application period (5-15min), and get the most negative slipe as the slope of drug.

In [ ]:
filterSize = 15 #sweeps
sweepLength = 10/60 #min
segmentSize = 10 #sweeps


In [ ]:
abfIDs = []

for abfFilePath in abfFilePaths:
    # get the tag-aligned raw data of holding current and time
    currentsRaw, timesRaw, abfID = abfTools.meanIhold(abfFilePath)
    abfIDs.append(abfID)
        
    # Get the smoothed holding current by the filterSize
    currentsSmooth, currentsSmoothTimes = statsTools.smoothY(currentsRaw, timesRaw, filterSize)
    
    # Separate the smoothed holding current into baseline period (0-5min) and drug application period (5-15min).
    baselineStartIndex, baselineEndIndex = statsTools.rangeIndex(currentsSmoothTimes, 2, 5)
    baselineCurrent = currentsSmooth[baselineStartIndex:baselineEndIndex]
    
    # Apply a single linear regression in the baseline period (0-5min) to get the slope of baseline.
    baselineSlope, baselineIntercept = slopeTools.getRegression(baselineCurrent, sweepLength)
    baselineTime = 3.5 #min
    
    # Generate multiple linear regression on the segregated data 
    currentSegs = statsTools.getMovingWindowSegments(currentsSmooth, segmentSize)
    segSlopes = slopeTools.getAllSegmentSlopes(currentSegs, segmentSize*sweepLength)
    tagTime = abfTools.getFirstTagTime(abfFilePath)
    slopeTimeOffset = segmentSize * sweepLength / 2-(tagTime-5)
    drugSlopeTimes = np.arange(len(segSlopes))*sweepLength + slopeTimeOffset
    
    # Find the most negative slope during drug application period (5-15min) as the slope of drug.
    drugSlopeMin = statsTools.rangeMin(segSlopes,drugSlopeTimes, 5, 15)
    drugSlopeMinIndex = segSlopes.index(drugSlopeMin)

    drugSlopeMinTime = drugSlopeTimes[drugSlopeMinIndex]
    
    # Plot the data and its slopes
  
    plt.figure()
    ax1 = plt.subplot(211)
    plotTools.plotCurrentOverTimeAndLabelPeakSlope(currentsRaw, timesRaw, drugSlopeMinTime, filterSize)
    plt.plot(currentsSmoothTimes, currentsSmooth, 'k')
    fittedBaslineX = np.linspace(0, 5)
    fittedBaslineY = fittedBaslineX*baselineSlope + baselineIntercept
    plt.plot(fittedBaslineX, fittedBaslineY, '--', color="red")
    plt.title(abfID)

    ax2 = plt.subplot(212, sharex=ax1)
    plotTools.plotSlopesOverTimeAndMarkpeakSlope(segSlopes, drugSlopeTimes, drugSlopeMinTime, drugSlopeMin)
    plt.plot(baselineTime, baselineSlope, 'b.', ms=15, alpha=.5)
    plt.show()
    
